In [23]:
# importing modules
import pandas as pd
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis256, Inferno256, Magma256
from bokeh.palettes import BrBG, PiYG, RdGy, RdYlGn, YlGnBu
from bokeh.models import LinearColorMapper, ColorBar

In [24]:
# shapefile column renaming
# districts_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')
# province_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')

In [25]:
# Reading the data
# read files
rheas= pd.read_csv('data/rheas/rheas_district_corrected.csv')
preharvest= pd.read_csv('data/preharvest/preharvest_cleaned.csv')

district_shp=gpd.read_file('data/shp/zmb_district_cleaned.shp')
country_shapefile=gpd.read_file('data/shp/gadm36_ZMB_0.shp')


In [26]:
e_list=[]
for d in list(preharvest.district.unique()):
    dfff=rheas.loc[rheas['district_b']==d]
    e_list.append(dfff)
    df2=pd.concat(e_list,ignore_index=True,axis=0)
rheas1=df2
rheas1['yield']=rheas1['yield'].round(decimals=2)
rheas1['yield_corrected']=rheas1['yield_corrected'].round(decimals=2)
rheas1['preharvest_yield']=rheas1['preharvest_yield'].round(decimals=2)
rheas1['yield_corrected'].min()

0.98

In [27]:
# Merge data into shapefiles
rheas_shp = pd.merge(district_shp, rheas1, on="district_b")

In [28]:
geo_source_rheas_shp = GeoJSONDataSource(geojson=rheas_shp.to_json())
country_geo_source = GeoJSONDataSource(geojson=country_shapefile.to_json())

In [29]:
color_mapper = LinearColorMapper(palette="RdBu10", low=3.97, high=4.05)
p1 = figure(title='RHEAS Model Before Bias Correction',
           outer_width=650, outer_height=600)

p1.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p1.patches('xs','ys', source=geo_source_rheas_shp,
          fill_color={'field': 'yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@name1'), ('yield', '@yield')])
p1.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p1.add_layout(color_bar, 'right')

In [30]:
color_mapper = LinearColorMapper(palette="RdBu10", low=0.94, high=4.04)
p2 = figure(title='Preharvest Surveys',
           outer_width=650, outer_height=600)

p2.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p2.patches('xs','ys', source=geo_source_rheas_shp,
          fill_color={'field': 'preharvest_yield', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@name1'), ('yield', '@preharvest_yield')])
p2.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p2.add_layout(color_bar, 'right')

In [31]:
color_mapper = LinearColorMapper(palette="RdBu10", low=0.98, high=4.06)
p3 = figure(title='RHEAS Model After Bias Correction',
           outer_width=650, outer_height=600)

p3.multi_line('xs','ys', source=country_geo_source,
            line_color='black', line_width=0.5)

p3.patches('xs','ys', source=geo_source_rheas_shp,
          fill_color={'field': 'yield_corrected', 'transform': color_mapper},
          fill_alpha=0.7, line_color='black', line_width=0.5)

hover = HoverTool(tooltips=[('District', '@name1'), ('yield', '@yield_corrected')])
p3.add_tools(hover)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=20, height=300,
                     border_line_color=None, location=(0,0), orientation='vertical')
p3.add_layout(color_bar, 'right')

In [32]:

# grid=gridplot([ p2, p3, p4, p1],ncols=2, width=650, height=600)
# grid = gridplot([[p2, p3], [None, p4]], width=650, height=600)
grid = gridplot([p1,p2,p3], ncols=3, width=650, height=600)
show(grid)

In [33]:
from bokeh.io import save
# # specify the output file name and location
save(grid, filename='../zambia.html', title='Maize Yield Maps')

C:\Users\kasit\AppData\Local\Temp\ipykernel_19012\1700210890.py:3: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(grid, filename='../zambia.html', title='Maize Yield Maps')


'd:\\RONO\\rheas\\maize_crop_modeling\\crop-modeling\\zambia.html'